In [33]:
import pandas as pd 
import seaborn as sns
import json
import math
from pandas.io.json import json_normalize  
import datetime as dt

TEST_MODE=True

def getAsFloat(elem):
    if (elem is not None) and (elem!='nan') and elem.replace('.','',1).isdigit():
        return float(elem)
    return 0
 
def getAsInt(elem):
    if (math.isnan(elem)):
        return 0
    return int(elem)

def getAsDate(elem):
    return pd.to_datetime(elem)

def getAveragePrice(row):
    if (row.surface_reelle_bati == 0):
        return 0;
    return int(row.valeur_fonciere/row.surface_reelle_bati);

def getConsidered(row):
    if (row.surface_reelle_bati == 0):
        return 0;
    if (row.surface_terrain > (2*row.surface_reelle_bati)):
        return 0;
    return 1;

def getParcelleSection(parcelle):
    return parcelle[5:10].strip('0')

def getLastRelevantPrice(x, y):
    if (x > 0):
        return x
    return y

def is_outlier(s):
    stdVal = s.std() * 3
    meanVal = s.mean()
    lower_limit = meanVal - stdVal
    upper_limit = meanVal + stdVal
    return ~s.between(lower_limit, upper_limit)


def computeDataFrameMeanOnYear(s, group, year):
    values = s[s['date_mutation'].dt.year == currentYear]
    values = values.groupby(group)['prix_m2'].mean().reset_index();
    return values.astype({group:'str'}).set_index(group)
    

def meanPriceDepartement(s, currentYear) :
    values = s[~s.groupby('code_departement')['prix_m2'].apply(is_outlier)]
    valuesLatest = computeDataFrameMeanOnYear(values, 'code_departement', currentYear)
    valuesOld = computeDataFrameMeanOnYear(values, 'code_departement', currentYear-1)
    latestKey = 'prix_m2_' + str(currentYear)
    oldKey = 'prix_m2_' + str(currentYear-1)

    values = pd.merge(valuesLatest, valuesOld, on='code_departement', 
                     suffixes=("_" +str(currentYear-1), "_" +str(currentYear)))
    values['prix_m2'] = values[[latestKey, oldKey]].apply(lambda row: row[latestKey] if row[latestKey] >0 else row[oldKey], axis=1)
    values['evolution'] = values[[latestKey, oldKey]].apply(lambda row: (row[latestKey]/row[oldKey])-1.0 if (row[latestKey] >0) and (row[oldKey] >0.0) else 0, axis=1)
    return values


def meanPriceCities(s) :
    values = s[~s.groupby('code_commune')['prix_m2'].apply(is_outlier)]      
    values = values.groupby('code_commune').aggregate({'code_departement': 'first',
                                                       'prix_m2': 'mean'}).reset_index();        
    return values.astype({'code_commune':'str'}).set_index('code_commune')


def meanPriceSections(s) :
    values = s[~s.groupby('code_section')['prix_m2'].apply(is_outlier)]
    values = values.groupby(['code_commune', 'code_section']).aggregate({'prix_m2': 'mean'}).reset_index();       
    return values.astype({'code_section':'str'}).set_index('code_section')



def meanPriceParcelles(s) :
    values = s.groupby(['code_commune', 'code_section', 'id_parcelle']).aggregate({'prix_m2': 'mean'}).reset_index();       
    return values.astype({'id_parcelle':'str'}).set_index('id_parcelle')


def getDepartementDetails() :
    with open("C:\\Users\\pifoyard\\Desktop\\NoCode\\MyCSV\\departements-100m.geojson", encoding='utf-8') as data_file:    
        data = json.load(data_file)  
    geo = json_normalize(data['features']) 
    geo.rename(columns={'properties.code': 'code_departement', 'properties.nom': 'departement', 'properties.region': 'code_region'}, inplace=True)
    geo.set_index('code_departement')    

    # Load regions
    with open("C:\\Users\\pifoyard\\Desktop\\NoCode\\MyCSV\\regions.json", encoding='utf-8') as data_file:    
        data = json.load(data_file)  
    regions = json_normalize(data) 
    regions.rename(columns={'nom': 'region', 'code': 'code_region'}, inplace=True)
    regions.set_index('code_region')

    # Remove first 0
    geo['code_region']=geo['code_region'].map(lambda x: x.lstrip('0'))
    geo['code_departement']=geo['code_departement'].map(lambda x: x.lstrip('0'))
    regions['code_region']=regions['code_region'].map(lambda x: x.lstrip('0'))

    # Join regions and geo info
    return geo.merge(regions, on='code_region', how='left').astype({'code_departement':'str'}).set_index('code_departement')


def buildDepartementsFile(consideredValues, filePath, currentYear):
    departements = meanPriceDepartement(consideredValues, currentYear)

    # Enrich with departement geozon
    if (TEST_MODE == False):
        details = getDepartementDetails()
        departements = details.join(departements)

    # round and set price to default value if required
    departements['prix_m2']=departements['prix_m2'].apply(getAsInt);
    
    # Save file
    departements.to_csv(filePath)

    
def buildCitiesFile(consideredValues, filePath):
    cities = meanPriceCities(consideredValues)

    # round and set price to default value if required
    cities['prix_m2']=cities['prix_m2'].apply(getAsInt);
    
    # Save file
    cities.to_csv(filePath)

def buildSectionsFile(consideredValues, filePath):
    sections = meanPriceSections(consideredValues)

    # round and set price to default value if required
    sections['prix_m2']=sections['prix_m2'].apply(getAsInt);
    
    # Save file
    sections.to_csv(filePath)    

    

def buildParcellesFile(consideredValues, filePath):
    sections = meanPriceParcelles(consideredValues)

    # round and set price to default value if required
    sections['prix_m2']=sections['prix_m2'].apply(getAsInt);
    
    # Save file
    sections.to_csv(filePath)    
        

# Load year values from: https://cadastre.data.gouv.fr/data/etalab-dvf/latest/csv/$YEAR/full.csv.gz
# Read source file
def extractFiles(files):   
    dataFrames = []
    for file in files:
        dataFrames.append(pd.read_csv( file, 
                                       sep=',',
                                       converters = {'lot1_surface_carrez': getAsFloat,
                                                     'lot2_surface_carrez': getAsFloat,
                                                     'lot3_surface_carrez': getAsFloat,
                                                     'lot4_surface_carrez': getAsFloat,
                                                     'lot5_surface_carrez': getAsFloat,
                                                     'surface_reelle_bati': getAsFloat,
                                                     'surface_terrain': getAsFloat,
                                                     'valeur_fonciere': getAsFloat,
                                                     'date_mutation': getAsDate
                                                    },
                                      encoding='utf-8'));
    data = pd.concat(dataFrames);

    # Build surface as max
    data['lot1_surface_carrez']=data[['lot1_surface_carrez',
                                    'lot2_surface_carrez',
                                    'lot3_surface_carrez',
                                    'lot4_surface_carrez',
                                    'lot5_surface_carrez']].sum(axis=1);
    data['surface_reelle_bati']=data[['lot1_surface_carrez','surface_reelle_bati']].max(axis=1);

    # Prepare set of mutations
    mutations = data.groupby('id_mutation').aggregate({ 'date_mutation': 'first',
                                                        'surface_reelle_bati': 'sum',
                                                        'surface_terrain': lambda x: x.unique().sum(),
                                                        'valeur_fonciere': 'max'}).reset_index();
    mutations['prix_m2']=mutations.apply(getAveragePrice, axis=1);
    mutations['considered_for_average']=mutations.apply(getConsidered, axis=1);

    # Prepare set of parcelles
    parcelles = data.groupby(['id_mutation', 'id_parcelle']).aggregate({'adresse_numero': 'first',
                                                                        'adresse_suffixe': 'first',
                                                                        'adresse_nom_voie': 'first',
                                                                        'adresse_code_voie': 'first',                                                        
                                                                        'code_postal': 'first',
                                                                        'code_commune': 'first',
                                                                        'nom_commune': 'first',
                                                                        'code_departement': 'first'}).reset_index();
    parcelles['code_section'] = parcelles['id_parcelle'].apply(getParcelleSection)

    # Complement parcelles with mutations
    mergedParcellesAndMutations = parcelles.join(mutations.set_index('id_mutation'), on= 'id_mutation')

    ######################################
    # MEAN CALCULATION

    # Remove rows not considered for average
    consideredValues = mergedParcellesAndMutations[mergedParcellesAndMutations.considered_for_average != 0]
    return consideredValues;
    
    
##############################################
##              MAIN                        ##
## Compute average price ignoring outliers  ##
##############################################
folder = "C:\\Users\\pifoyard\\Desktop\\NoCode\\MyCSV\\"
currentYear=2019
consideredValues = extractFiles([folder + "full2019" + ".csv"]);
consideredValues[consideredValues['code_departement']==57].head()

#buildDepartementsFile(consideredValues, folder + "departements.csv", currentYear)
#buildCitiesFile(consideredValues, folder + "cities.csv")
#buildSectionsFile(consideredValues, folder + "sections.csv")
#buildParcellesFile(consideredValues, folder + "parcelles.csv")
    

,id_mutation,id_parcelle,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,code_section,date_mutation,surface_reelle_bati,surface_terrain,valeur_fonciere,prix_m2,considered_for_average
